In [1]:
__author__ = "Lucia Seitz"
__copyright__ = "Copyright 2017, AI Research, Data Technology Centre, Volkswagen Group"
__credits__ = ["Lucia Seitz, Karolina Stosio"]
__license__ = "MIT"

import glob
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import misc
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA
from sklearn.neighbors import KNeighborsClassifier
import scipy as sc
%matplotlib inline

In [2]:
import edward as ed
import os
import tensorflow as tf

from edward.models import Bernoulli, Normal
from edward.util import Progbar
from observations import mnist
from scipy.misc import imsave
from tensorflow.contrib import slim

In [3]:
def generator(array, batch_size):
    """Generate batch with respect to array's first axis."""
    start = 0    # pointer to where we are in iteration
    while True:
        stop = start + batch_size
        diff = stop - array.shape[0]
        if diff <= 0:
            batch = array[start:stop]
            start += batch_size
        else:
            batch = np.concatenate((array[start:], array[:diff]))
            start = diff
        batch = batch.astype(np.float32) / 255.0    # normalize pixel intensities
        batch = np.random.binomial(1, batch)    # binarize images
        yield batch


def generative_network(z):
    """Generative network to parameterize generative model. It takes
    latent variables as input and outputs the likelihood parameters.
    logits = neural_network(z)
    """
    with slim.arg_scope([slim.conv2d_transpose],
                                            activation_fn=tf.nn.elu,
                                            normalizer_fn=slim.batch_norm,
                                            normalizer_params={'scale': True}):
        print('generative')
        print(z)
        net = tf.reshape(z, [M, 1, 1, d])
        print(net)
        net = slim.conv2d_transpose(net, 128, 4, stride=4)
        print(net)
        net = slim.conv2d_transpose(net, 64, 5, stride=2)
        print(net)
        net = slim.conv2d_transpose(net, 32, 5, stride=2)
        print(net)
        net = slim.conv2d_transpose(net, 16, 5, stride=2)
        print(net)
        net = slim.conv2d_transpose(net, 8, 5, stride=4)
        print(net)
        net = slim.conv2d_transpose(net, 4, 5, stride=2, activation_fn=None)
        print(net)
        net = slim.flatten(net)
        print(net)
        return net


def inference_network(x):
    """Inference network to parameterize variational model. It takes
    data as input and outputs the variational parameters.
    loc, scale = neural_network(x)
    """
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                                            activation_fn=tf.nn.elu,
                                            normalizer_fn=slim.batch_norm,
                                            normalizer_params={'scale': True}):
        print('inference')
        print(x)
        #net = tf.reshape(x, [M, 28, 28, 1])
        net = tf.reshape(x, [M, 256, 256, 4])
        print(net)
        net = slim.conv2d(net, 8, 5, stride=2)
        print(net)
        net = slim.conv2d(net, 16, 5, stride=3)
        print(net)
        net = slim.conv2d(net, 32, 5, stride=3)
        print(net)
        net = slim.conv2d(net, 64, 5, stride=2)
        print(net)
        net = slim.conv2d(net, 128, 5, padding='VALID')
        print(net)
        net = slim.dropout(net, 0.9)
        print(net)
        net = slim.flatten(net)
        print(net)
        params = slim.fully_connected(net, d * 2, activation_fn=None)
        print('params',net)

    loc = params[:, :d]
    scale = tf.nn.softplus(params[:, d:])
    print('loc',loc)
    print('scale',scale)
    return loc, scale

In [4]:
# PATH = '/home/karolina/Documents/VW/synthetic_data'
# images = glob.glob(PATH+'/*/*.png')
# images.sort()
# train_data = np.array([misc.imresize(misc.imread(im),20) for im in images[:100]])
# names = [im.split('/')[-2] for im in images]
# n_set = list(set(names))
# numbers = [[i]*names.count(n_set[i]) for i in range(len(n_set))]
# img_x, img_y, img_chan = train_data.shape[1:]

In [5]:
# train_data.shape

In [6]:
ed.set_seed(42)

data_dir = "/tmp/data"
out_dir = "/tmp/out"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
ed.set_seed(42)
M = 128    # batch size during training
d = 10     # latent dimension

In [7]:
# DATA. MNIST batches are fed at training time.
(x_train, _), (x_test, _) = mnist(data_dir)
x_train_generator = generator(x_train, M)
#x_train_generator = generator(train_data, M)

# MODEL
z = Normal(loc=tf.zeros([M, d]), scale=tf.ones([M, d]))
logits = generative_network(z)
x = Bernoulli(logits=logits)

# INFERENCE
x_ph = tf.placeholder(tf.int32, [M, 256*256*4])
#x_ph = tf.placeholder(tf.int32, [M, img_x, img_y, img_chan])
loc, scale = inference_network(tf.cast(x_ph, tf.float32))
qz = Normal(loc=loc, scale=scale)

# Bind p(x, z) and q(z | x) to the same placeholder for x.
data = {x: x_ph}
inference = ed.KLqp({z: qz}, data)
optimizer = tf.train.AdamOptimizer(0.01, epsilon=1.0)
inference.initialize(optimizer=optimizer)

hidden_rep = tf.sigmoid(logits)

tf.global_variables_initializer().run()


generative
RandomVariable("Normal_2/", shape=(128, 10), dtype=float32)
Tensor("Reshape:0", shape=(128, 1, 1, 10), dtype=float32)
Tensor("Conv2d_transpose/Elu:0", shape=(128, 4, 4, 128), dtype=float32)
Tensor("Conv2d_transpose_1/Elu:0", shape=(128, 8, 8, 64), dtype=float32)
Tensor("Conv2d_transpose_2/Elu:0", shape=(128, 16, 16, 32), dtype=float32)
Tensor("Conv2d_transpose_3/Elu:0", shape=(128, 32, 32, 16), dtype=float32)
Tensor("Conv2d_transpose_4/Elu:0", shape=(128, 128, 128, 8), dtype=float32)
Tensor("Conv2d_transpose_5/BatchNorm/batchnorm/add_1:0", shape=(128, 256, 256, 4), dtype=float32)
Tensor("Flatten/Reshape:0", shape=(128, 262144), dtype=float32)
inference
Tensor("Cast:0", shape=(128, 262144), dtype=float32)
Tensor("Reshape_1:0", shape=(128, 256, 256, 4), dtype=float32)
Tensor("Conv/Elu:0", shape=(128, 128, 128, 8), dtype=float32)
Tensor("Conv_1/Elu:0", shape=(128, 43, 43, 16), dtype=float32)
Tensor("Conv_2/Elu:0", shape=(128, 15, 15, 32), dtype=float32)
Tensor("Conv_3/Elu:0", s

In [8]:
# ### TRAINING

# n_epoch = 100
# n_iter_per_epoch = x_train.shape[0] // M
# for epoch in range(1, n_epoch + 1):
#     print("Epoch: {0}".format(epoch))
#     avg_loss = 0.0

#     pbar = Progbar(n_iter_per_epoch)
#     for t in range(1, n_iter_per_epoch + 1):
#         pbar.update(t)
#         x_batch = next(x_train_generator)
#         info_dict = inference.update(feed_dict={x_ph: x_batch})
#         avg_loss += info_dict['loss']

#     # Print a lower bound to the average marginal likelihood for an
#     # image.
#     avg_loss = avg_loss / n_iter_per_epoch
#     avg_loss = avg_loss / M
#     print("-log p(x) <= {:0.3f}".format(avg_loss))

#     # Visualize hidden representations.
#     images = hidden_rep.eval()
#     for m in range(M):
#         imsave(os.path.join(out_dir, '%d.png') % m, images[m].reshape(28, 28))
